Knihovna `langdetect` není bůhvíco. Část českých projevů identifikuje jako slovinštinu, němčina v citacích vede k označení projevu za německý… Pro současný účel rozpoznávání – rozlišit češtinu a slovenštinu pro přesnější vývoje výskytů různých českých slov – to je s přimhouřenýma očima a několika ručně zbastlenými pravidly ok. Pomohlo by i rozpoznávat nezávisle začátek a konec textu a pak nějak řešit nesoulad. Další možnost: nechat si vracet seznam jazyků s pravděpodobnostmi a pravidla pro dočištění postavit na něm. Nebo zkrátka: najít lepší nástroj.

In [73]:
import os
import string
import polars as pl
import altair as alt

pl.Config.set_tbl_rows(300)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [74]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    (pl.col('datum').dt.year() >= 1918)
    & 
    (~pl.col('komora_komplet').str.contains('(?i)senát')) 
    & 
    (~pl.col('komora_komplet').str.contains('(?i)společn'))
).sort(
    by="datum"
).collect()

In [75]:
from langdetect import detect
from langdetect import detect_langs

In [76]:
def jazyk(retezec):
    try:
        return detect(retezec)
    except Exception as e:
        print(retezec)
        print(e)
        print("***")
        return None

In [77]:
def jazyky(retezec):
    try:
        return detect_langs(retezec)
    except Exception as e:
        print(retezec)
        print(e)
        print("***")
        return None

In [78]:
jazyk("Přerušuji jednání do 15.40 hodin. (Jednání bylo přerušeno v 15.23 hodin.)")

'cs'

## Trvá detekce delšího textu déle?

In [80]:
test = df.filter(pl.col("soubor") == "1929ns_ps_stenprot_014schuz_s014007.htm").filter(pl.col("poradi") == 3).select(pl.col("text")).item()

In [81]:
%%timeit

jazyky(test)

6.96 ms ± 385 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [82]:
%%timeit

jazyky(test[0:300])

6.73 ms ± 396 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
%%timeit

jazyky(test[0:100])

6.75 ms ± 155 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
jazyky("Pak bych vás chtěla poprosit, abyste mi začali posílat své požadavky na ubytování, kapacity jsou omezené.")

[cs:0.9999948593605235]

Netrvá, není nutno ořezávat.

## Kolik času to vezme?

In [87]:
len(df)

729386

In [88]:
velikost_vzorku = int(len(df) / 1000)
velikost_vzorku

729

In [89]:
vzorek = df.sample(velikost_vzorku)

In [90]:
%%timeit
vzorek.with_columns(pl.col("text").map_elements(jazyk, return_dtype=pl.String).alias("jazyk"))

The slowest run took 5.09 times longer than the fastest. This could mean that an intermediate result is being cached.
12.2 s ± 5.1 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Naostro

In [92]:
df = df.select(
    pl.col("text")
).unique(
).with_columns(
    pl.col("text").map_elements(jazyk, return_dtype=pl.String).alias("jazyk")
)

221/13.
No features in text.
***
681.
No features in text.
***
85.
No features in text.
***
11.30?
No features in text.
***
)
No features in text.
***

No features in text.
***
511.
No features in text.
***
69/6.
No features in text.
***
724.
No features in text.
***
199
No features in text.
***
20. 6. 2025.
No features in text.
***
.
No features in text.
***
63.
No features in text.
***
31. 12. 2000.
No features in text.
***
,
No features in text.
***
[ ].
No features in text.
***


In [219]:
df.write_parquet(
    os.path.join("data_raw","projevy_jazyky_raw.parquet")
)

In [123]:
jazyky("Děkuji panu poslanci Stindlovi.")

[hr:0.7142826470336449, cs:0.28571417796221255]

In [125]:
df.filter(pl.col("jazyk") == "sl")

text,jazyk
str,str
"""Slova se ujme pan kolega Langer. Připraví se pan kolega Ransdorf.""","""sl"""
"""Pan posl. dr Otakar Vašek.""","""sl"""
"""Děkuji. Pan poslanec Exner.""","""sl"""
"""Slova se ujme pan kolega Maixner, připraví se pan kolega Vrcha.""","""sl"""
"""za člena N. S. Zíku jest navržen člen N. S. Karel Svoboda;""","""sl"""
"""Pan poslanec Dušek.""","""sl"""
"""Slova se ujme pan kolega Špaček, připraví se pan kolega Vrzal.""","""sl"""
"""Slova se ujme kolega Ortman, připraví se kolega Čapek.""","""sl"""
"""Slovo má pan dr. Kordač. (Potlesk).""","""sl"""


In [117]:
df.group_by("jazyk").len().sort(by="len",descending=True)

jazyk,len
str,u32
"""cs""",679021
"""sk""",14505
"""sl""",2963
"""hr""",768
"""de""",301
"""en""",263
"""cy""",230
"""id""",200
"""tl""",135


In [130]:
df.filter(pl.col("jazyk") == "de").sample(10)

text,jazyk
str,str
"""Ne, ihned.""","""de"""
"""B/142. Nesouhlas. (Ministr nesouhlas.)""","""de"""
"""B9.""","""de"""
"""B38.""","""de"""
"""Klub poslanců ""Vereinigter parlam. Klub des Bundes der Landwirte und der Deutschen Arbeits- und Wirtschaftsgemeinschaft"" vyslal do výboru soc.-politic…","""de"""
"""D28.""","""de"""
"""62. Nesouhlas. (Ministr: Nesouhlas.)""","""de"""
"""""Ich gelobe, der Čechoslovakischen Republik treu zu sein und die Gesetze zu beobachten sowie mein Mandat nach bestem Wissen und Gewissen auszuüben.""""","""de"""
"""Dre Wintře Lve!""","""de"""


In [132]:
df.filter(pl.col("jazyk") == "en").sample(10)

text,jazyk
str,str
"""Josef Pták, in memoriam.""","""en"""
"""Dále C5. Souhlas. (Ministr: Souhlas.)""","""en"""
"""B/115 - souhlas. (Ministr: Souhlas.)""","""en"""
"""B/96. Nesouhlas. (Ministr: Nesouhlas.)""","""en"""
"""47. Nesouhlas. (Ministr: Nesouhlas.)""","""en"""
"""264. Souhlas. (Ministr: Souhlas.)""","""en"""
"""154 - nesouhlas. (Ministr: Souhlas.)""","""en"""
"""32. Souhlas. (Ministr: Souhlas.)""","""en"""
"""Pan posl.Jozef Horváth.""","""en"""


In [207]:
rucni_docisteni = df.with_columns(
    pl.col("text").str.len_bytes().alias("delka")
).with_columns(
    pl.when(
        pl.col("jazyk").is_in(['cs','sk'])
    ).then(
        pl.col('jazyk')
    ).when(
        pl.col('jazyk') == 'sl'
    ).then(
        pl.lit('cs')
    ).otherwise(
        pl.lit(None)
    ).alias(
        'jazyk'
    )
).drop(
    'delka'
)

rucni_docisteni

text,jazyk
str,str
"""Děkuji panu předsedovi Kalouskovi. Ptám se, jestli se ještě někdo hlásí do obecné rozpravy. Není tomu tak, končím tedy obecnou rozpravu. Ptám se na zá…","""cs"""
"""(12.50 hodin) (pokračuj Filip) Jestli to vůbec přijmou a jestli to bude pro ně prospěšné. Nechtěl jsem mluvit dlouho, ale považoval jsem za potřebné t…","""cs"""
"""Vážený pane předsedající, dámy a pánové, tady padlo několik otázek. A já se přiznám, že jsem si pořád ještě nezvykl na to, že když se dělá nějaký záko…","""cs"""
"""Děkuji. Jenom v rychlosti doplním ještě, minulá koaliční vláda, kde jsme byli jako KDU-ČSL součástí, snižovala daně. Pro pana předsedu z SPD Fialu dop…","""cs"""
"""Kromě bodů uvedených zpravodajem navrhuji hlasovat odděleně ještě o bodech 2 a 5.""","""cs"""
"""207. Pozměňovací návrh posl. Kučery a soudr.: V § 105, odst. 1, lit. c) budiž v druhé větě za slova: ""manželku (manžela)"" vloženo: ""nebo družku (druha…","""cs"""
"""Děkuji a poprosím dalšího z řádně přihlášených, což je pan poslanec Igor Hendrych, který bude interpelovat řádně omluveného ministra Blažka.""","""cs"""
"""Já vám děkuji, pane zpravodaji. Teď by byl prostor pro protinávrhy, ale ještě předtím bude prostor vystoupit k hlasování předchozímu. Poprosím.""","""cs"""
"""Chtěl bych říci, že souhlasím s panem ministrem, že se nám vracejí choroby chudých dětí, šíří se TBC. Z tohoto hlediska je podle mne potřebné, aby se …","""cs"""


In [227]:
df.with_columns(
    pl.col("text").str.len_bytes().alias("delka")
).group_by("jazyk").agg(pl.col('delka').median()).sort(by="delka")

jazyk,delka
str,f64
null,4.0
"""ca""",4.0
"""vi""",5.0
"""da""",6.0
"""lv""",9.5
"""it""",11.0
"""tr""",12.0
"""so""",12.0
"""sw""",12.0


In [211]:
df.group_by("jazyk").len().sort(by="len",descending=True)

jazyk,len
str,u32
"""cs""",679021
"""sk""",14505
"""sl""",2963
"""hr""",768
"""de""",301
"""en""",263
"""cy""",230
"""id""",200
"""tl""",135


In [213]:
rucni_docisteni.group_by("jazyk").len().sort(by="len",descending=True)

jazyk,len
str,u32
"""cs""",681984
"""sk""",14505
null,2685


In [199]:
df.with_columns(
    pl.col("text").str.len_bytes().alias("delka")
).filter(
    ~pl.col('jazyk').is_in(['cs','sk'])
).sort(
    by='delka',descending=True
).filter(pl.col('delka').is_between(1000,9150)).sample(10)

text,jazyk,delka
str,str,u32
"""Poslanec František Bublan. (Slibuji.) Poslanec Jan Bureš. (Slibuji.) Poslanec Václav Cempírek. (Slibuji.) Poslanec Josef Cogan. (Slibuji.) Poslanec Ja…","""sl""",1154
"""Koháček Petr. (Slibuji.) Kohlíček Jaromír. (Slibuji.) Kopecký Robert. (Slibuji.) Korbel Ladislav. (Slibuji.) Kořistka Zdeněk. (Slibuji.) Kostrhun Jan.…","""sl""",1825
"""(14.30 hodin) Maštálka Jiří: Slibuji. Melčák Miloš: Slibuji. Mencl Václav: Slibuji. Michalík Alfréd: Slibuji. Mikuta Josef: Slibuji. Mládek Jan: Slib…","""sl""",2071
"""Pan poslanec Jan Hofmann. (Slibuji.) Pan poslanec Radim Holiš. (Slibuji.) Pan poslanec Jiří Horák. (Slibuji.) (Poslanec Horák z KDU-ČSL za Jihomoravsk…","""sl""",2315
"""(14.20 hodin) Konečná Kateřina. (Slibuji.) Koníček Vladimír. (Slibuji.) Korytář Karel. (Slibuji.) Kováčik Pavel. (Slibuji.) Krajíček Miroslav. (Slibu…","""sl""",1812
"""Doktor Michal. (Slibuji.) Doležal Vladimír. (Slibuji.) Dostál Pavel. (Slibuji.) Dostálová Kateřina. (Slibuji.) Drda Jiří. (Slibuji.) Dundáčková Eva. (…","""sl""",1424
"""(14.10 hodin) Dědič František. (Slibuji.) Dimitrov Kosta. (Slibuji.) Dlouhý Vladimír. (Slibuji.) Doktor Michal. (Slibuji.) Dolejš Jiří. (Slibuji.) Do…","""sl""",1288
"""Ransdorf Miloslav. (Slibuji.) Recman Svatomír. (Slibuji.) Rozehnal Aleš. (Slibuji.) Rujbrová Zuzka. (Slibuji.) Sehnalová Olga. (Slibuji.) Sehoř Karel.…","""sl""",1689
"""(14.20 hodin) Grebeníček Miroslav: Slibuji. Grospič Stanislav: Slibuji. Gross Stanislav: Slibuji. Gruntová Jitka: Slibuji. Grůza Jan: Slibuji. Hanuš …","""sl""",1509


In [218]:
rucni_docisteni.write_parquet(
    os.path.join("data","projevy_jazyky.parquet")
)

In [96]:
df.sample(50)

text,jazyk
str,str
"""Děkuji za slovo. Dobré dopoledne, paní předsedající, pane premiére, kolegové a členové vlády. Počkal jsem si v té rozpravě trpělivě, byl jsem řádně př…","""cs"""
"""Prosím nyní pana poslance Jiřího Paroubka ještě do obecné rozpravy.""","""cs"""
"""Mou povinností bude při hlasování o jednotlivých pozměňovacích návrzích minimální výši vládní rozpočtové rezervy pečlivě hlídat. Pozměňovací návrhy, k…","""cs"""
"""Vážený pane předsedo, vážená vládo, vážené dámy a páni poslanci, smlouva mezi Českou republikou a Belgickým královstvím o zamezení dvojího zdanění a z…","""cs"""
"""Pan kolega Igor Svoják.""","""sl"""
"""Já bych ráda, paní předsedající, vážené dámy a pánové, podpořila návrh pana poslance Jičínského, protože vychází z dikce jednacího řádu. My opravdu má…","""cs"""
"""Paní předsedající, paní a pánové, dovolte mi... až se uklidní situace... jenom velmi krátký komentář. Ten rozdíl je tak veliký, že nemohlo jít o náhod…","""cs"""
"""Do rozpravy je jako první s přednostním právem přihlášen pan místopředseda Sněmovny Okamura. Prosím, máte slovo.""","""cs"""
"""Děkuji. Kdo souhlasí s tímto návrhem, ať zvedne ruku! 14. Kdo je proti? 23. Ani tento návrh nebyl přijat.""","""cs"""
